# **BUSINESS CASE 2: SIEMENS ADVANTA SALES FORECASTING**  


## 🎓 Master’s Program in Data Science & Advanced Analytics 
**Nova IMS** | March 2025   
**Course:** Business Cases with Data Science

## 👥 Team **Group A**  
- **Alice Viegas** | 20240572  
- **Bernardo Faria** | 20240579  
- **Dinis Pinto** | 20240612  
- **Daan van Holten** | 20240681
- **Philippe Dutranoit** | 20240518

## 📊 Goal of the notebook

in this notebook we make the build and test the models for forcasting

# Imports

In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import itertools
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from prophet import Prophet


from Model_Functions import *

import importlib
imported_module = importlib.import_module("Model_Functions")
importlib.reload(imported_module)


<module 'Model_Functions' from '/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Model_Functions.py'>

In [47]:
sales_agg1 = pd.read_csv('../data/sales_agg1.csv').set_index('DATE')
sales_agg3 = pd.read_csv('../data/sales_agg3.csv').set_index('DATE')
sales_agg4 = pd.read_csv('../data/sales_agg4.csv').set_index('DATE')
sales_agg5 = pd.read_csv('../data/sales_agg5.csv').set_index('DATE')
sales_agg6 = pd.read_csv('../data/sales_agg6.csv').set_index('DATE')
sales_agg8 = pd.read_csv('../data/sales_agg8.csv').set_index('DATE')
sales_agg9 = pd.read_csv('../data/sales_agg9.csv').set_index('DATE')
sales_agg11 = pd.read_csv('../data/sales_agg11.csv').set_index('DATE')
sales_agg12 = pd.read_csv('../data/sales_agg12.csv').set_index('DATE')
sales_agg13 = pd.read_csv('../data/sales_agg13.csv').set_index('DATE')
sales_agg14 = pd.read_csv('../data/sales_agg14.csv').set_index('DATE')
sales_agg16 = pd.read_csv('../data/sales_agg16.csv').set_index('DATE')
sales_agg20 = pd.read_csv('../data/sales_agg20.csv').set_index('DATE')
sales_agg36 = pd.read_csv('../data/sales_agg36.csv').set_index('DATE')

# extract the selected features list build in the feature selection step
selected_features_1 = pd.read_csv('../data/selected_features_1.csv')['selected_features'].tolist()
selected_features_3 = pd.read_csv('../data/selected_features_3.csv')['selected_features'].tolist()
selected_features_4 = pd.read_csv('../data/selected_features_4.csv')['selected_features'].tolist()
selected_features_5 = pd.read_csv('../data/selected_features_5.csv')['selected_features'].tolist()
selected_features_6 = pd.read_csv('../data/selected_features_6.csv')['selected_features'].tolist()
selected_features_8 = pd.read_csv('../data/selected_features_8.csv')['selected_features'].tolist()
selected_features_9 = pd.read_csv('../data/selected_features_9.csv')['selected_features'].tolist()
selected_features_11 = pd.read_csv('../data/selected_features_11.csv')['selected_features'].tolist()
selected_features_12 = pd.read_csv('../data/selected_features_12.csv')['selected_features'].tolist()
selected_features_13 = pd.read_csv('../data/selected_features_13.csv')['selected_features'].tolist()
selected_features_14 = pd.read_csv('../data/selected_features_14.csv')['selected_features'].tolist()
selected_features_16 = pd.read_csv('../data/selected_features_16.csv')['selected_features'].tolist()
selected_features_20 = pd.read_csv('../data/selected_features_20.csv')['selected_features'].tolist()
selected_features_36 = pd.read_csv('../data/selected_features_36.csv')['selected_features'].tolist()


# load the market data
df_market = pd.read_csv('../data/Market_data_predicted.csv').set_index('date')
test_set = pd.read_csv('../Data/Case2_Test Set Template.csv', sep=';')
print(sales_agg1.head())
df_market.head()



              Sales_EUR
DATE                   
2018-10-01  36098918.79
2018-11-01   5140760.00
2018-12-01  37889612.12
2019-01-01  27728148.35
2019-02-01  34793163.53


,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_SHP250,MAB_ELE_PRO392,MAB_ELE_SHP392,MAB_ELE_PRO756,MAB_ELE_SHP826,MAB_ELE_SHP840,RohiBASEMET1000_org,RohiMETMIN1000_org,RohiNATGAS1000_org,RohCOPPER1000_org,WKLWEUR840_org,PRI27380_org,PRI27276_org,PRO28756_org,PRO28392_org,PRO271000_org,PRO27756_org,PRO27392_org
date,,,,,,,,,,,,,,,,,,,,
2004-02-01,16.940704,16.940704,83.458866,109.334010,110.495272,86.074496,77.766367,90.442749,54.039811,48.747945,87.076974,36.623832,1.2646,93.020027,93.230453,96.177481,110.890034,91.390440,68.677657,106.161262
2004-03-01,23.711852,23.711852,106.168192,140.884616,144.686166,86.038258,77.829225,90.580357,54.666162,49.256157,87.192705,39.931055,1.2262,93.540268,93.335678,96.250927,141.176853,91.511498,68.452548,140.288741
2004-04-01,24.435235,24.435235,92.007646,105.853579,102.655769,85.997063,77.904642,90.751805,54.872715,49.423751,91.379923,39.134854,1.1985,93.852425,93.440903,96.330211,105.648765,91.666790,68.203892,106.271197
2004-05-01,23.708115,23.708115,85.696486,101.864777,100.305285,85.951483,77.993606,90.961426,51.230356,46.468392,99.044520,36.278433,1.2007,93.852425,93.546127,96.413549,101.990361,91.861457,67.936322,101.608710
2004-06-01,27.009138,27.009138,106.641482,120.332920,119.616380,85.902640,78.096995,91.213203,52.876331,47.803913,98.636267,35.657380,1.2138,93.956467,93.440903,96.499291,122.136575,92.100289,67.655734,116.655248


In [49]:
test_set

,Month Year,Mapped_GCK,Sales_EUR
0,Mai 22,#3,NaN
1,Jun 22,#3,NaN
2,Jul 22,#3,NaN
3,Aug 22,#3,NaN
4,Sep 22,#3,NaN
...,...,...,...
135,Okt 22,#1,NaN
136,Nov 22,#1,NaN
137,Dez 22,#1,NaN
138,Jan 23,#1,NaN


In [50]:
month_map = {
    "Jan": 1,
    "Feb": 2,
    "Mai": 3,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Okt": 10,
    "Nov": 11,
    "Dez": 12
}

# Apply conversion safely
test_set['Month Year'] = test_set['Month Year'].apply(parse_month_year)

# Preparation 

## Create the lag market data feature needed for the model

In [51]:
df_market = create_lag_features(df_market, 6)
df_market.head()

/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Model_Functions.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Model_Functions.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Not

,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_SHP250,MAB_ELE_PRO392,MAB_ELE_SHP392,MAB_ELE_PRO756,MAB_ELE_SHP826,MAB_ELE_SHP840,RohiBASEMET1000_org,RohiMETMIN1000_org,...,RohiNATGAS1000_org_lag_6,RohCOPPER1000_org_lag_6,WKLWEUR840_org_lag_6,PRI27380_org_lag_6,PRI27276_org_lag_6,PRO28756_org_lag_6,PRO28392_org_lag_6,PRO271000_org_lag_6,PRO27756_org_lag_6,PRO27392_org_lag_6
date,,,,,,,,,,,,,,,,,,,,,
2004-02-01,16.940704,16.940704,83.458866,109.334010,110.495272,86.074496,77.766367,90.442749,54.039811,48.747945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-01,23.711852,23.711852,106.168192,140.884616,144.686166,86.038258,77.829225,90.580357,54.666162,49.256157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-04-01,24.435235,24.435235,92.007646,105.853579,102.655769,85.997063,77.904642,90.751805,54.872715,49.423751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-05-01,23.708115,23.708115,85.696486,101.864777,100.305285,85.951483,77.993606,90.961426,51.230356,46.468392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-06-01,27.009138,27.009138,106.641482,120.332920,119.616380,85.902640,78.096995,91.213203,52.876331,47.803913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Created lagged sales features needed for the model

In [ ]:
sales_agg1 = create_lag_features(sales_agg1, 10)
sales_agg3 = create_lag_features(sales_agg3, 10)
sales_agg4 = create_lag_features(sales_agg4, 10)
sales_agg5 = create_lag_features(sales_agg5, 10)
sales_agg6 = create_lag_features(sales_agg6, 10)
sales_agg8 = create_lag_features(sales_agg8, 10)
sales_agg9 = create_lag_features(sales_agg9, 10)
sales_agg11 = create_lag_features(sales_agg11, 10)
sales_agg12 = create_lag_features(sales_agg12, 10)
sales_agg13 = create_lag_features(sales_agg13, 10)
sales_agg14 = create_lag_features(sales_agg14, 10)
sales_agg16 = create_lag_features(sales_agg16, 10)
sales_agg20 = create_lag_features(sales_agg20, 10)
sales_agg36 = create_lag_features(sales_agg36, 10)

## Define the parameter grid to run

In [ ]:
xgboost_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7, 1],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 0.3],
    'reg_lambda': [0, 0.1, 0.3]
}

prophet_param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'yearly_seasonality': [True, False],
    'weekly_seasonality': [True, False],
    'daily_seasonality': [True, False],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'changepoint_prior_scale': [0.01, 0.1, 0.5]
}

# Product 1

In [ ]:
# selecte only the features that are selected in the feature selection step
df_market_1 = df_market[selected_features_1]
df_market_1.head()

/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Model_Functions.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Model_Functions.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag_{lag}'] = df[col].shift(lag)
/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Not

,PRO27756_org_lag_6,MAB_ELE_SHP250_lag_4,MAB_ELE_SHP156,PRO271000_org_lag_5
date,,,,
2004-02-01,NaN,NaN,16.940704,NaN
2004-03-01,NaN,NaN,23.711852,NaN
2004-04-01,NaN,NaN,24.435235,NaN
2004-05-01,NaN,NaN,23.708115,NaN
2004-06-01,NaN,83.458866,27.009138,NaN


## Xgboost

In [ ]:
X_train, X_val, y_train, y_val = train_val_split(sales_agg1, df_market_1, train_size = 0.8)

X_val.info() 

xgboost_best_model_1, xgboost_best_params_1, xgboost_best_rmse_1 = model_evaluation_XGboost(X_train,
                                                                                             y_train, 
                                                                                             X_val, y_val, 
                                                                                             xgboost_param_grid)

print("Best Params:", xgboost_best_params_1)
print("Validation RMSE:", xgboost_best_rmse_1)



<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 2021-08-01 to 2022-04-01
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Sales_EUR_lag_1       9 non-null      float64
 1   Sales_EUR_lag_2       9 non-null      float64
 2   Sales_EUR_lag_3       9 non-null      float64
 3   Sales_EUR_lag_4       9 non-null      float64
 4   Sales_EUR_lag_5       9 non-null      float64
 5   Sales_EUR_lag_6       9 non-null      float64
 6   Sales_EUR_lag_7       9 non-null      float64
 7   Sales_EUR_lag_8       9 non-null      float64
 8   Sales_EUR_lag_9       9 non-null      float64
 9   Sales_EUR_lag_10      9 non-null      float64
 10  PRO27756_org_lag_6    9 non-null      float64
 11  MAB_ELE_SHP250_lag_4  9 non-null      float64
 12  MAB_ELE_SHP156        9 non-null      float64
 13  PRO271000_org_lag_5   9 non-null      float64
dtypes: float64(14)
memory usage: 1.1+ KB
Params: {'n_estimators': 100

## Prophet

In [ ]:
prophet_best_model_1, prophet_best_params_1, prophet_best_rmse_1 = model_evaluation_prophet(X_train,
                                                                                             y_train, 
                                                                                             X_val, 
                                                                                             y_val, 
                                                                                             prophet_param_grid)

print("Best Params:", prophet_best_params_1)

print("Validation RMSE:", prophet_best_rmse_1)

14:15:22 - cmdstanpy - INFO - Chain [1] start processing
14:15:23 - cmdstanpy - INFO - Chain [1] done processing
14:15:23 - cmdstanpy - INFO - Chain [1] start processing
14:15:23 - cmdstanpy - INFO - Chain [1] done processing
14:15:23 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.01} => RMSE: 3357351.5431
Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.1} => RMSE: 3356405.4135


14:15:23 - cmdstanpy - INFO - Chain [1] done processing
14:15:23 - cmdstanpy - INFO - Chain [1] start processing
14:15:23 - cmdstanpy - INFO - Chain [1] done processing
14:15:23 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.5} => RMSE: 3854203.3100
Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.01} => RMSE: 69941639.1351


14:15:23 - cmdstanpy - INFO - Chain [1] done processing
14:15:23 - cmdstanpy - INFO - Chain [1] start processing


...

Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.1} => RMSE: 3264371.3843
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.5} => RMSE: 3853678.4100


14:18:05 - cmdstanpy - INFO - Chain [1] done processing
14:18:05 - cmdstanpy - INFO - Chain [1] start processing
14:18:06 - cmdstanpy - INFO - Chain [1] done processing
14:18:06 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.01} => RMSE: 3259864.6103
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.1} => RMSE: 3264371.3843


14:18:06 - cmdstanpy - INFO - Chain [1] done processing


Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.5} => RMSE: 3853678.4100
Best Params: {'seasonality_mode': 'additive', 'yearly_seasonality': False, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.1}
Validation RMSE: 2866894.4721454247
